In [7]:
import ee
import geemap
import geemap.colormaps as cm
import ipywidgets as widgets
from bqplot import pyplot as plt
from ipyleaflet import WidgetControl

Map = geemap.Map()
Map.add_basemap('HYBRID')
Map.setCenter(76.8006, 30.6238, 14)
Map

Map(center=[30.6238, 76.8006], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chi…

In [8]:
style = {'description_width': 'initial'}
output_widget = widgets.Output(layout={'border': '1px solid black'})

nd_options = ['Vegetation Index (NDVI)', 
              'Modified Water Index (MNDWI)',
              'Soil Index (NDSI)',
              'Customized']
nd_indices = widgets.Dropdown(options=nd_options, value='Modified Water Index (MNDWI)', description='Normalized Difference Index:', style=style)

year_widget = widgets.IntSlider(min=2017, max=2021, value=2021, description='Selected year:', width=400, style=style)

nd_options = ['Vegetation Index (NDVI)', 
              'Modified Water Index (MNDWI)',
              'Soil Index (NDSI)',
              'Customized']

first_band = widgets.Dropdown(
    description='1st band:',
    options=['B2', 'B3','B4','B8', 'B11', 'B12'],
    value='B3',
    style=style
)

second_band = widgets.Dropdown(
    description='2nd band:',
    options=['B2', 'B3','B4','B8', 'B11', 'B12'],
    value='B11',
    style=style
)

nd_threshold = widgets.FloatSlider(
    value=0,
    min=-1,
    max=1,
    step=0.01,
    description='Threshold:',
    orientation='horizontal',
    style=style
)

nd_color = widgets.ColorPicker(
    concise=False,
    description='Color:',
    value='blue',
    style=style
)

def nd_index_change(change):
    if nd_indices.value == 'Vegetation Index (NDVI)':
        first_band.value = 'B8'
        second_band.value = 'B4' 
    elif nd_indices.value == 'Modified Water Index (MNDWI)':
        first_band.value = 'B3'
        second_band.value = 'B11'   
    elif nd_indices.value == 'Soil Index (NDSI)':
        first_band.value = 'B11'
        second_band.value = 'B8'        
    elif nd_indices.value == 'Customized':
        first_band.value = None
        second_band.value = None
        
nd_indices.observe(nd_index_change, names='value')

submit = widgets.Button(
    description='Submit',
    button_style='primary',
    tooltip='Click me',
    style=style
)

full_widget = widgets.VBox([
    widgets.HBox([year_widget,nd_threshold]),
    widgets.HBox([nd_indices, first_band, second_band]),
    submit
])

full_widget

In [9]:
def submit_clicked(b):
     with output_widget:
        output_widget.clear_output()
        print('Computing...')
        Map.default_style = {'cursor': 'wait'}
        
        try:
            #def submit_clicked(b):
            band1 = first_band.value
            band2 = second_band.value
            selected_year = year_widget.value
            threshold = nd_threshold.value
            #palette = nd_color.value
            def maskS2clouds(image):
                qa = image.select('QA60')
                #  # Bits 10 and 11 are clouds and cirrus, respectively.
                cloudBitMask = 1 << 10
                cirrusBitMask = 1 << 11
                # Both flags should be set to zero, indicating clear conditions.
                mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
                    .And(qa.bitwiseAnd(cirrusBitMask).eq(0))
                return image.updateMask(mask).divide(10000)
            geometry= ee.Geometry.Polygon([[76.79911551271984,30.6168475776271],
                            [76.79911551271984,30.62644958134403],
                            [76.78280768190929,30.62644958134403],
                            [76.78280768190929,30.6168475776271]])
            S2_SR = ee.ImageCollection('COPERNICUS/S2_SR').filterBounds(geometry)\
                    .filterDate('2021-01-01', '2021-08-06')\
                    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)) \
                               .map(maskS2clouds)\
                    .select(['B2','B3','B4','B8','B11','B12'])
            def addVI(image):
                VI = image.normalizedDifference([band1, band2]).rename('VI')
                return image.addBands(VI)
            S2_SR = S2_SR.map(addVI)
            image= S2_SR.select('VI').toBands()
            recent_S2 = ee.Image(S2_SR.sort('system:time_start', False).first())
            palette = cm.palettes.ndvi
            ndviVis = {
                'min': 0.0,
                'max': 1.0,
                'palette': palette
                 }
            VI=recent_S2.select('VI')
            #Map.setCenter(76.8006, 30.6238, 14)
            #Map.set_plot_options(add_marker_cluster=False,position='bottomright',overlay=False,max_height=250,max_width=800)
            # To Draw features on a particular layer
            Map.addLayer(image,{}, 'Timeseries of Sentinel VI',opacity=.1)
            roi1 = ee.FeatureCollection(Map.draw_features)
            ndvi_img=VI.clip(roi1)       
            Map.addLayer(ndvi_img,ndviVis,'VI_image')
            
                           
        except Exception as e:
            print(e)
            print('An error occurred during computation.') 
        Map.default_style = {'cursor': 'default'}
            
submit.on_click(submit_clicked)